# Demonstration of Creating and Running a NATS input file from IFF data
## Read in IFF Data using PARA-ATM

In [2]:
from paraatm.io import iff, gnats, utils
from paraatm.io.iff import read_iff_file


#iff_fname = 'IFF_SFO_ASDEX_ABC123.csv'
iff_fname = 'IFF_SFO+ASDEX_20190511_080104_86221.csv'

iff_data =  read_iff_file(iff_fname,record_types=[2,3,4,8])
print(iff_data[3].columns)
print(len(iff_data[3]))

Index(['recType', 'time', 'fltKey', 'bcnCode', 'cid', 'Source', 'msgType',
       'callsign', 'recTypeCat', 'latitude', 'longitude', 'altitude',
       'significance', 'coord1Accur', 'coord2Accur', 'altAccur', 'tas',
       'heading', 'rocd', 'altQualifier', 'altIndicator', 'trackPtStatus',
       'leaderDir', 'scratchPad', 'msawInhibitInd', 'assignedAltString',
       'controllingFac', 'controllingSeg', 'receivingFac', 'receivingSec',
       'activeContr', 'primaryContr', 'kybrdSubset', 'kybrdSymbol', 'adsCode',
       'opsType', 'airportCode', 'trackNumber', 'tptReturnType', 'modeSCode'],
      dtype='object')
1030410


## Clean callsign list of UNKN, nan, 1200, and ground operations vehicles

In [5]:
#Note not necessary and will produce an error if run 
#with sample file
callsigns = iff_data[3].callsign.unique()
cs = callsigns.tolist()
cs.remove('UNKN') # remove 'UNKN'
cs = [x for x in cs if (str(x) != 'nan')] #remove nan
cs = [x for x in cs if not x.startswith('OPS')] #remove ground operation vehicles
cs.remove('1200')

## Start NATS Simulation in the background

In [6]:
from gnats_gate_to_gate import GateToGate
natsSim = GateToGate()

## Create FlightPlanSelector object

In [7]:
from FlightPlanSelector import FlightPlanSelector
dirPath = natsSim.DIR_share
fpath = dirPath + '/tg/trx/TRX_07132005_noduplicates_crypted'
f=FlightPlanSelector(natsSim,fname=fpath)

## Generate NATS TRX flightplans

In [10]:
import os
from iff_functions import (get_departure_airport_from_iff,
                           get_arrival_airport_from_iff,
                           get_departure_gate_and_rwy_from_iff,
                           get_arrival_gate_and_rwy_from_iff,
                           random_airport_gate_and_rwy,
                           check_if_flight_has_departed,
                           check_if_flight_landing_in_dataset)

iffBaseAirport= "KSFO"

#Filenames to write modified trx and mfl files to. Need full path because GNATS wrapper changes the directory to the GNATS directory.
home_env = os.environ.get('HOME')
trx_dir = '/home/edecarlo/para-atm-collection/miscellaneous/gnats-fpgen'
results_dir = '/home/edecarlo/para-atm-collection/miscellaneous/gnats-fpgen'

trx_fname = '/iff_to_gnats_geo'
mfl_file= trx_dir+trx_fname+'_mfl.trx'
trx_file = trx_dir+trx_fname+'.trx'
results_file = results_dir+trx_fname+'.csv'

if os.path.exists(trx_file):
    os.remove(trx_file)
    print('Previously generated trx file removed...')

if os.path.exists(mfl_file):
    os.remove(mfl_file)
    print('Previously generated mfl file removed...')

Previously generated trx file removed...
Previously generated mfl file removed...


In [11]:
import time
import numpy as np
from jpype import JPackage

TRACK_TIME = time.time()
clsGeometry = JPackage('com').osi.util.Geometry

for i,callsign in enumerate(cs[101:110]):
    flightInAir = check_if_flight_has_departed(iff_data,callsign,natsSim,iffBaseAirport)
    flightLandingInData = check_if_flight_landing_in_dataset(iff_data,callsign,natsSim,iffBaseAirport)
    
    arrivalAirport = ''
    departureAirport=''
    arrivalRwy = ''
    arrivalGate = ''
    departureRwy =''
    departureGate = ''
    result_generated = ''

    if flightInAir and flightLandingInData:
        #Get departure airport. If none is known in the IFF+ASDEX file (i.e., it is not the airport whose name is in the iff_fname), then set departure airport as closest airport to the first lat/lon in the dataset. In the future, would like to use IFF_USA to determine departureAirport in this case
        print (callsign, i)
        arrivalAirport = 'KSFO'
        departureAirport = get_departure_airport_from_iff(iff_data,callsign,natsSim,arrivalAirport=iffBaseAirport,flmap=f.flmap)
        #departureAirport = 'KEWR'
        arrivalGate,arrivalRwy= get_arrival_gate_and_rwy_from_iff(iff_data,callsign,natsSim,arrivalAirport)
        result_generated = f.generate(4, departureAirport, arrivalAirport, "", arrivalGate, "", arrivalRwy)
        print(result_generated)
        

        lat = iff_data[3].loc[iff_data[3].callsign==callsign,'latitude'].iloc[0]
        lon = iff_data[3].loc[iff_data[3].callsign==callsign,'longitude'].iloc[0]
        latstr = clsGeometry.convertLatLonDeg_to_degMinSecString(str(lat))
        lonstr = clsGeometry.convertLatLonDeg_to_degMinSecString(str(lon))
        elev = np.max([iff_data[3].loc[iff_data[3].callsign==callsign,'altitude'].iloc[0]/100.,100.])
        spd = iff_data[3].loc[iff_data[3].callsign==callsign,'tas'].iloc[0]
        hdg = iff_data[3].loc[iff_data[3].callsign==callsign,'heading'].iloc[0]
        aircraftType = 'B744'
        
    if  not flightInAir and not flightLandingInData:
        print (callsign, i)
        departureAirport = 'KSFO'
        arrivalAirport = get_arrival_airport_from_iff(iff_data,callsign,natsSim,departureAirport,f.flmap)
        #arrivalAirport = 'KEWR'
        departureGate,departureRwy = get_departure_gate_and_rwy_from_iff(iff_data,callsign,natsSim,departureAirport) #doesn't exist
        arrivalGate,arrivalRwy = random_airport_gate_and_rwy(natsSim,arrivalAirport)
        result_generated = f.generate(1, departureAirport, arrivalAirport, departureGate, arrivalGate, departureRwy, arrivalRwy)
        print(result_generated)
        
        airportInstance = natsSim.airportInterface.select_airport(departureAirport)
        elev = airportInstance.getElevation()/100.0  
        lat = airportInstance.getLatitude()
        lon = airportInstance.getLongitude()
        latstr = clsGeometry.convertLatLonDeg_to_degMinSecString(str(lat))
        lonstr = clsGeometry.convertLatLonDeg_to_degMinSecString(str(lon))
        aircraftType = 'B744'
        spd = 0
        hdg = 28
   
    if result_generated:
        TRACK_TIME += 10
        with open(trx_file,'a') as trxFile:
            tstr = '%s %s %s %.4f %.4f %d %.2f %d %s %s' % ('TRACK',callsign,aircraftType,float(latstr),float(lonstr),spd,elev,hdg,'ZOA','ZOA46')
            trxFile.write('%s %d' % ('TRACKTIME', TRACK_TIME) + '\n')
            trxFile.write( tstr + '\n')
            trxFile.write('    FP_ROUTE ' + result_generated[0] + '\n\n')

        with open(mfl_file,'a') as mflFile:
            mflFile.write(callsign + ' ' + '330' + '\n')

UAL780 0
('KSFO.<{"id": "Gate_G_100"}, {"id": "Ramp_07_007"}, {"id": "Txy_B_B5"}, {"id": "Txy_B5_001"}, {"id": "Txy_A_B4"}, {"id": "Txy_A_B3"}, {"id": "Txy_A_B2"}, {"id": "Txy_A_B1"}, {"id": "Txy_A_016"}, {"id": "Txy_Q1_001"}, {"id": "Txy_B_002"}, {"id": "Txy_B_K"}, {"id": "Txy_B_D"}, {"id": "Txy_E_B"}, {"id": "Txy_E_001"}, {"id": "Rwy_03_005"}, {"id": "Txy_E_002"}, {"id": "Txy_E_003"}, {"id": "Rwy_04_007"}, {"id": "Txy_E_004"}, {"id": "Txy_C_E"}, {"id": "Txy_E_V"}, {"id": "Txy_V_004"}, {"id": "Rwy_01_007"}, {"id": "Txy_V_003"}, {"id": "Txy_V_002"}, {"id": "Rwy_02_008"}, {"id": "Txy_V_001"}, {"id": "Txy_L_V"}, {"id": "Txy_L_E"}, {"id": "Txy_L_010"}, {"id": "Txy_L_011"}, {"id": "Rwy_02_011"}>.RW19L.WESLA4.SAC.J32.BAM.J32.CZI.J82.RAP.J158.ABR..CESNA..SSM..YCF..TULEG..RKA..HNK.SHAFF7.I11.RW11 .<{"id": "Rwy_03_102"}, {"id": "Rwy_03_002"}, {"id": "Txy_Z_025"}, {"id": "Txy_K_015"}, {"id": "Txy_Y_K"}, {"id": "Txy_Y_016"}, {"id": "Rwy_01_004"}, {"id": "Txy_Y_L"}, {"id": "Txy_Y_012"}, {"id": "T

TypeError: cannot unpack non-iterable NoneType object

In [ ]:
data=natsSim(trx_name=trx_file,mfl_name=mfl_file)['trajectory']
natsSim.write_output(results_file)
natsSim.cleanup()

In [ ]:
from paraatm.plotting import plot_trajectory
plot_trajectory(data, output_notebook=True, plot_width=600, plot_height=400)

In [14]:
result_generated

''

In [16]:
if not result_generated:
    print('not result generated is True')

not result generated is True
